In [ ]:
#--------------------------------------------------------------------------------
# Example: Sequencing SignalFlow with isobar
# 
# In this example, a Cymbal patch is created at initialisation.
# Each event triggers the patch, causing its ASREnvelope to be re-triggered.
#--------------------------------------------------------------------------------

from isobar import *
from signalflow import *

graph = AudioGraph()
output_device = SignalFlowOutputDevice(graph)
timeline = Timeline(120, output_device=output_device, start=True)

In [ ]:
class Cymbal (Patch):
    def __init__(self, hpf: float = 5000):
        super().__init__()
        hpf = self.add_input("hpf", hpf)
        noise = WhiteNoise()
        noise = SVFilter(noise, "high_pass", hpf)
        envelope = ASREnvelope(0, 0, 0.1)
        output = noise * envelope * 0.25
        self.set_output(output)
        
        # set_trigger_node() determines which node will be re-triggered
        # when the patch receives a trigger() event
        self.set_trigger_node(envelope)

In [ ]:
cymbal = Cymbal()
cymbal.play()

In [ ]:
# re-trigger the cymbal manually
cymbal.trigger()

In [ ]:
timeline.schedule({
    "type": "trigger",
    "patch": cymbal,
    "duration": PLoop(PTri(16, 0.5, 0.1)),
    "params": {
        "hpf": PWhite(1000, 10000)
    }
}, name="cymbal")

In [ ]:
timeline.clear()
graph.clear()